In [ ]:
import pandas as pd
import numpy as np

path = 'E:/BaiduYunDownload/DC_2/'

importance = pd.read_csv(path+'feature_importance.csv',encoding = 'gb18030')
data = pd.read_csv(path+'data0219.csv',encoding = 'gb18030',index_col=None)
data = data.fillna(0)

for index,i in enumerate(data.columns):
    if 'cost_sum' in i:
        data[i+str(index)] = data[i]/data['costpos_sum']
for index,i in enumerate(data.columns):
    if 'count' in i:
        data[i+str(index)] = data[i]/data['costpos_count']

In [ ]:
from sklearn import metrics
from sklearn import cross_validation
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from imblearn.over_sampling import RandomOverSampler

data = data.fillna(0)
train_data = data[data['TT']=='train']
x = train_data.drop(['TT','student_id','grant_amount','class_rank',],axis = 1)
y = train_data['grant_amount']

test_data = data[data['TT']=='test']
test_x = test_data.drop(['TT','student_id','grant_amount','class_rank',],axis = 1)
test_y = test_data['grant_amount']

os =  RandomOverSampler(ratio=1.0)
X_overs, y_overs = os.fit_sample(x, y)

X_overs = pd.DataFrame(X_overs)
X_overs.columns = test_x.columns
y_overs = pd.DataFrame(y_overs)
y_overs.columns = ['grant_amount']
y_overs = y_overs['grant_amount']

#data_train, data_test, target_train, target_test = cross_validation.train_test_split(X_overs, y_overs)

In [ ]:
new_data1 = X_overs[importance['feature'][0:101]]
new_data2 = test_x[importance['feature'][0:101]]

new_data3 = pd.concat([new_data1,new_data2],axis = 0)
new_data3 = new_data3.reset_index().drop('index',axis = 1)

etc_en = pd.read_csv(path+'etc_en.csv',index_col=None)
etc_gini = pd.read_csv(path+'etc_gini.csv',index_col=None)
rf_en = pd.read_csv(path+'rf_en.csv',index_col=None)
rf_gini = pd.read_csv(path+'rf_gini.csv',index_col=None)
xgb1 = pd.read_csv(path+'xgb.csv',index_col=None)
GBDT = pd.read_csv(path+'xgb.csv',index_col=None)
ada = pd.read_csv(path+'xgb.csv',index_col=None)
bag = pd.read_csv(path+'xgb.csv',index_col=None)

stacking_result = pd.concat([etc_en,etc_gini,rf_en,rf_gini,xgb1,GBDT,ada,bag],axis = 1)

In [ ]:
new = stacking_result#pd.concat([new_data3,stacking_result],axis = 1)
new_train = new.iloc[0:37300,:]
new_test = new.iloc[37300::,:]

data_train, data_test, target_train, target_test = cross_validation.train_test_split(new_train, y_overs)

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.02,
 n_estimators=800,
 max_depth=3,
 min_child_weight=1,
 gamma = 0.01,
 subsample=0.85,
 colsample_bytree=0.85,
 objective= 'multi:softmax',
 seed = 10,
 nthread=8,
)

xgb1.fit(data_train, target_train,eval_metric='auc')    

train_pred = xgb1.predict(data_test)
    
print "\nModel Report"
#print "AUC : %.4g" % metrics.roc_auc_score(target_test, dtrain_predprob)
print "f1 : %.4g" % metrics.f1_score(target_test, train_pred,average = 'macro')

In [ ]:
y_pred = xgb1.predict(new_test)
submission = pd.DataFrame()
submission['studentid']  = test_data['student_id']
submission['subsidy'] = y_pred

submission.to_csv(path+'result0220.csv',sep = ',', index=False)
submission['subsidy'].value_counts()

In [ ]:
import  matplotlib.pyplot as plt  
%matplotlib inline
###画图显示中文
plt.rcParams['font.family'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

feat_imp = pd.Series(xgb1.booster().get_fscore()).sort_values(ascending=True)
feat_imp[-40:-1].plot(kind='barh', title='Feature Importances',figsize=(6,12))
plt.ylabel('Feature Importance Score')

golden_feature = feat_imp[-40:-1].index
#importance = pd.DataFrame(xgb1.booster().get_fscore().items(), columns=['feature','importance']).sort('importance', ascending=False)
#importance.to_csv(path+'feature_importance.csv',sep = ',', index=False,encoding = 'gb18030')